In [17]:
import pandas as pd;
import numpy as np;
import math
import random
from sklearn.cluster import KMeans
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping,TensorBoard

In [18]:

#loading human observed datasets
featureData=pd.read_csv("HumanObserved-Features-Data_HO.csv")
diffPairsHumanData=pd.read_csv("diffn_pairs_HO.csv");
samePairsHumanData=pd.read_csv("same_pairs_HO.csv");

#loading gsc dataset
gscFeatureData=pd.read_csv("GSC-Features_GSC.csv")
diffPairsGSCData=pd.read_csv("diffn_pairs_GSC.csv")
samePairsGSCData=pd.read_csv("same_pairs_GSC.csv")
#number of clusters or the number of basis fucntions
NUMBER_OF_CLUSTERS=15;
#the method creates a dictionary
def createDictionary(imageNumberList,listOfListOfFeatures):
    imageFeatureMap={};
    for i in range(0,len(imageNumberList)):
        imageId=str(imageNumberList[i][0]);
        #print(imageId)
        imageFeatureMap[imageId]=listOfListOfFeatures[i];
    return imageFeatureMap;
#the method returns dictionary with imageId and the corresponding features
def getFeatureDictionary(featureData, isHumanObserverd):
    if isHumanObserverd:
        featureData.drop(featureData.columns[0],axis=1,inplace=True)
        
    onlyFeatureData=featureData.iloc[:,1:]
    onlyImageNumbers=featureData.iloc[:,0:1]
    listOfListOfFeatures=onlyFeatureData.values.tolist()
    imageNumberList=onlyImageNumbers.values.tolist()
    return createDictionary(imageNumberList,listOfListOfFeatures);

#fetching the imageid-feature dictionary for human data
imageFeatureMap=getFeatureDictionary(featureData,True);
#fetching the imageid-feature dictionary for gsc data
imageFeatureMapGSC=getFeatureDictionary(gscFeatureData,False);

diffPairsHumanSample=diffPairsHumanData.sample(n=samePairsHumanData.shape[0])

#diffPairsGscSample=diffPairsGSCData.sample(n=samePairsGSCData.shape[0])
diffPairsGscSample=diffPairsGSCData.sample(n=10000);
samePairsGscSample=samePairsGSCData.sample(n=10000);
#extracting image pairs and their targets from the data file
diffPairsHumanDataList=diffPairsHumanSample.iloc[:,0:3].values.tolist()
samePairsHumanDataList=samePairsHumanData.iloc[:,0:3].values.tolist();
diffPairsGSCDataList=diffPairsGscSample.iloc[:,0:3].values.tolist();
#samePairsGSCDataList=samePairsGSCData.iloc[:,0:3].values.tolist();
samePairsGSCDataList=samePairsGscSample.iloc[:,0:3].values.tolist();

In [19]:
# would be preparing data as per the scheme
#scheme-1:concatenation of features|scheme-2:subtraction of features
#returning 2-d array with features(as per the scheme) and the last column as the target
def prepareDataByScheme(pairedData,imageFeatureMap,toBeConcattenated):
    featureArrayList=[];
    for i in range(0,len(pairedData)):
        imageId1=pairedData[i][0];
        imageId2=pairedData[i][1];
        target=pairedData[i][2];
        feature1ArrayList=imageFeatureMap[imageId1];
        feature2ArrayList=imageFeatureMap[imageId2];
        #if toBeConcattenated true then concatenating else subtracting
        if toBeConcattenated:
            modifiedFeature=feature1ArrayList + feature2ArrayList
        else:
            modifiedFeature=[feature1ArrayList[i] - feature2ArrayList[i] for i in range(0,len(feature1ArrayList))];
        
        modifiedFeatureList=modifiedFeature
        modifiedFeatureList.append(target);
        featureArrayList.append(modifiedFeatureList);
    return np.array(featureArrayList);

In [36]:
  


def initializeModel(data):
    model=Sequential()
    model.add(Dense(100, input_dim=data.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dense(100,kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model;f
 
    
    
def oprateNeuralNetwork(model,trainingFeatureData,validationFeatureData,trainingTargets,validationTargets):   
    validation_data_split = 0.15
    num_epochs = 25
    model_batch_size = 128
    tb_batch_size = 32
    early_patience = 100
    trainValData=np.concatenate((trainingFeatureData,validationFeatureData));
    trainValLabels=np.concatenate((trainingTargets,validationTargets));
    history = model.fit(trainValData
                        , trainValLabels
                        , validation_split=validation_data_split
                        , epochs=num_epochs
                        , batch_size=model_batch_size
                                               )
    return history;



# def printKerasGraphs(history):  
#     get_ipython().run_line_magic('matplotlib', 'inline')
#     df = pd.DataFrame(history.history)
#     df.plot(subplots=True, grid=True, figsize=(10,15))

def evaluateNeuralNetwork(model,testFeatureData,testTargets):
    loss,metric=model.evaluate(testFeatureData,testTargets,batch_size=128)
    return metric;

In [37]:
TRAINING_DATA_PERCENT=0.8;
VALIDATION_DATA_PERCENT=0.1
TEST_DATA_PERCENT=0.1

# this method would be merging samewriter comparison data and the different write comparison data
def mergeData(sameWriterData,differentWriterData):
    sameWrLen=sameWriterData.shape[0];
    indices=np.random.randint(0,differentWriterData.shape[0],size=sameWrLen)    
    randomData=differentWriterData[indices];
    #randomData=differentWriterData[0:sameWriterData.shape[0],:];
    return np.concatenate((sameWriterData,randomData));

#method would be checking if any column is 0 in the merged data(2-D array), if yes then would be returning the indexes of the zero columns
def verifyingAllZeroColumns(mergedData):
    return np.argwhere(np.all(mergedData[..., :] == 0, axis=0));
#dadding noise
def modifyingMergedData(mergedData,zeroValueColumns):
    for i in range(0,mergedData.shape[1]):
        mergedData[i][i]=mergedData[i][i]+0.0001;
    return mergedData;
#partitioning the data into the traing,validation and test sets    
def partitionData(rawData):
    trainingLen=math.ceil(rawData.shape[0]*TRAINING_DATA_PERCENT);
    testingLen=math.ceil(rawData.shape[0]*TEST_DATA_PERCENT);
    validationLen=math.ceil(rawData.shape[0]*VALIDATION_DATA_PERCENT);
    return rawData[0:trainingLen,:],rawData[trainingLen:validationLen+trainingLen,:],rawData[validationLen+trainingLen:validationLen+trainingLen+testingLen];

In [38]:
#returning the variance vector, which has a variance across each and every feature column 
def generateVarianceVector(trainingFeatureData):
    varVect=[];
    vect=[];
    columns=trainingFeatureData.shape[1];
    rows=trainingFeatureData.shape[0];
    for k in range(0,trainingFeatureData.shape[1]):
        vect=trainingFeatureData[:,k:k+1]    
        varVect.append(np.var(vect));
    return varVect;
#returning the covariance matrix for the training data
def generatingCoVarianceMatrix(varianceVector,length):
    #covarianceMatrix=[[0 for i in range(0,length)] for j in range(0,length)];
    covarianceMatrix=np.zeros((length,length))
    for i in range(0,length):
        covarianceMatrix[i][i]=varianceVector[i];
    return np.dot(200,covarianceMatrix);#multiplying by 200, to broaden the deviation curve

In [39]:
#generating the value of the gaussian basis fucntion
def getScalar(inputDiff,bigSigmaInverse):
    halfProduct=np.matmul(inputDiff,bigSigmaInverse);
    fullproduct=np.matmul(halfProduct,inputDiff.transpose());
    return math.exp(-1*(fullproduct/2));
#generating the design matrix for the data, using gaussian radial fucntion as the basis
def generateDesignMatrix(bigSigmaInverse,centroidList,mergedFeatureData):
    designMatrix = np.zeros((mergedFeatureData.shape[0],centroidList.shape[0]))
    for i in range(0,mergedFeatureData.shape[0]):
        inputSample=mergedFeatureData[i];
        for j in range(0,centroidList.shape[0]):
                inputDiff=np.subtract(inputSample,centroidList[j].transpose());
                product=getScalar(inputDiff,bigSigmaInverse);
                designMatrix[i][j]=product;
    return designMatrix;

In [40]:
#for computing the outputs, based on the predicted weights
def GetValTest(VAL_PHI,W):
    Y = np.dot(W,np.transpose(VAL_PHI))
    return Y

#computing the loss for the Erms
def getSquaredDifferenceError(VAL_TEST_OUT,valDataAct):
    sum = 0.0
    t=0
    accuracy = 0.0
    counter = 0
    val = 0.0
    for i in range (0,len(VAL_TEST_OUT)):
        sum = sum + math.pow((valDataAct[i].item() - VAL_TEST_OUT[i]),2)
        if(int(np.around(VAL_TEST_OUT[i], 0)) == valDataAct[i]):
                counter+=1;
    return sum,counter;
#calculating Erms
def GetErms(VAL_TEST_OUT,ValDataAct):
    error,accuracy=getSquaredDifferenceError(VAL_TEST_OUT,ValDataAct);
    accuracyPercent=(accuracy/len(VAL_TEST_OUT) * 100);
    return math.sqrt(error/len(VAL_TEST_OUT));

In [41]:
#gradient Descent Solution-:
NUMBER_OF_TRANSACTIONS=400;
La=2;
learningRate=0.01;

In [42]:
def GetWeightsClosedForm(PHI, T, Lambda):
    #regularizing the data, in order to avoid the problem of overfitting
    Lambda_I = np.identity(len(PHI[0]))
    for i in range(0,len(PHI[0])):
        Lambda_I[i][i] = Lambda
    PHI_T       = np.transpose(PHI)
    
    PHI_SQR     = np.dot(PHI_T,PHI)
    PHI_SQR_LI  = np.add(Lambda_I,PHI_SQR)
    PHI_SQR_INV = np.linalg.inv(PHI_SQR_LI)
    INTER       = np.dot(PHI_SQR_INV, PHI_T)
    print(INTER.shape);
    print(T.shape)
    W           = np.dot(INTER, T)
    ##print ("Training Weights Generated..")
    return W

def applyGradientDescent(trainingTargets,validationTargets,testTargets,designMatrixTraining,designMatrixValidation,designMatrixTesting):
    #W=GetWeightsClosedForm(designMatrixTraining,trainingTargets,0.01);
    W_LIST= [0 for i in range(0,NUMBER_OF_CLUSTERS)]
    W=np.asarray(W_LIST);
    #W= np.dot(220, W)
    L_Erms_TR=[];
    L_Erms_Val=[];
    L_Erms_Test=[];
    for i in range(0,NUMBER_OF_TRANSACTIONS):
            Delta_E_D  = -np.dot((trainingTargets[i].item() - np.dot(W.transpose(),designMatrixTraining[i])),designMatrixTraining[i]);
            La_Delta_E_W  = np.dot(La,W);
            Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
            Delta_W       = -np.dot(learningRate,Delta_E)
            #updating the value of the weights
            W_T_Next      = W + Delta_W
            #assigning the weights to the new wieght vector
            W         = W_T_Next
            #-----------------TrainingData Accuracy---------------------#
            TR_TEST_OUT   = GetValTest(designMatrixTraining,W_T_Next) 
            Erms_TR       = GetErms(TR_TEST_OUT,trainingTargets)
            L_Erms_TR.append(float(Erms_TR))

            #-----------------ValidationData Accuracy---------------------#
            VAL_TEST_OUT  = GetValTest(designMatrixValidation,W_T_Next) 
            Erms_Val      = GetErms(VAL_TEST_OUT,validationTargets)
            L_Erms_Val.append(float(Erms_Val))

            #-----------------TestingData Accuracy---------------------#
            TEST_OUT= GetValTest(designMatrixTesting,W_T_Next) 
            Erms_Test = GetErms(TEST_OUT,testTargets)
            L_Erms_Test.append(float(Erms_Test))
    print ('----------Gradient Descent Solution--------------------')
    print ("E_rms Training   = " + str(np.around(min(L_Erms_TR),5)))
    print ("E_rms Validation = " + str(np.around(min(L_Erms_Val),5)))
    print ("E_rms Testing    = " + str(np.around(min(L_Erms_Test),5)))
    return W

In [43]:
def crossEntropy(weights,targets,trainingData):
    h=calculateSigmoid(weights,trainingData);
    y=targets
    loss=np.multiply(-1 * y,np.log(h))-np.multiply((1-y),np.log(1-h));
    cost=np.mean(loss)
    erms=getErmsForLogistic(cost,y.shape[0]);

def getErmsForLogistic(loss,dataSize):
    return np.sqrt(loss*2/dataSize);   
#computing the accuracy for logistic regression
def computeAccuracy(weights,data,targets):
    _Y=calculateSigmoid(data,weights);
    counter=0;
    _Y=np.around(_Y);
    for i in range(0,_Y.shape[0]):
        if targets[i].item()==_Y[i].item():
            counter=counter+1;
    return (counter/targets.shape[0]) * 100;     
#applying gradient desecnt, to compute the weights for training data
def logisticRegression(trainingData,targets):
    W_LIST= np.zeros([trainingData.shape[1], 1])
    W=np.asarray(W_LIST);
    print(W.shape)
    for i in range(0,400):
        h=calculateSigmoid(trainingData, W)
        X=trainingData.T
        delW=np.dot(X,(h-targets))/targets.shape[0]
        W-=.01*delW;
    return W;
        
def findSigmoid(z):  
    return 1/(1+np.exp(-z));
#computing the sigmoid for given weights and data(establishing the genesis equation)
def calculateSigmoid(data,weights):
    dproduct=np.dot(data, weights);
    y=findSigmoid(dproduct);
    return y

In [44]:
def deriveData(data,featureLength):
    featureData=data[:,0:featureLength]
    targets=data[:,featureLength:featureLength+1]
    return featureData,targets
#clustering data using k means
def clusterData(trainingFeatureData):
    kmeans = KMeans(n_clusters=NUMBER_OF_CLUSTERS, random_state=0).fit(trainingFeatureData)
    return kmeans.cluster_centers_

In [45]:

def projectKnob(num):
    if num==1:# case for human data set(subtraction)
        differentWriterDataSet=prepareDataByScheme(diffPairsHumanDataList,imageFeatureMap,False);
        sameWriterDataSet=prepareDataByScheme(samePairsHumanDataList,imageFeatureMap,False);
    elif num==2:# case for human data set(concatenation)
        differentWriterDataSet=prepareDataByScheme(diffPairsHumanDataList,imageFeatureMap,True);
        sameWriterDataSet=prepareDataByScheme(samePairsHumanDataList,imageFeatureMap,True);
    elif num==3:# case for gsc data set(subtraction) 
        differentWriterDataSet=prepareDataByScheme(diffPairsGSCDataList,imageFeatureMapGSC,False);
        sameWriterDataSet=prepareDataByScheme(samePairsGSCDataList,imageFeatureMapGSC,False)
    else:# case for gsc data set(concatenation)
        differentWriterDataSet=prepareDataByScheme(diffPairsGSCDataList,imageFeatureMapGSC,True);
        sameWriterDataSet=prepareDataByScheme(samePairsGSCDataList,imageFeatureMapGSC,True)
    #merging data(same writer data with the different writer data )
    mergedData=mergeData(sameWriterDataSet,differentWriterDataSet);
    print('--------merge data dim-------',mergedData.shape);
    #validating for zero columns(as they can make the covariance matrix singular)
    zeroValueColumns=verifyingAllZeroColumns(mergedData);
    #removing the zero columns
    #mergedData=modifyingMergedData(mergedData,zeroValueColumns);
    #shuffling or randomizing the merged data
    np.random.shuffle(mergedData);
    print('--------random merge data dim-------',mergedData.shape);
    #partitioning the data into 3 sets(training,validationData,testData)
    trainingData,validationData,testData=partitionData(mergedData);
    print('--------training data dim-------',trainingData.shape);
    print('--------validation data dim-------',validationData.shape);
    print('--------testData data dim-------',testData.shape);
    
    featureLength=mergedData.shape[1]-1;
    #deriving features data, targets data for training set.
    trainingFeatureData,trainingTargets=deriveData(trainingData,featureLength);
    print('--------training feature data dim-------',trainingFeatureData.shape);
    print('--------training targets dim-------',trainingTargets.shape);   
   #deriving features data, targets data for validation set.
    validationFeatureData,validationTargets=deriveData(validationData,featureLength);
    print('--------training feature data dim-------',validationFeatureData.shape);
    print('--------training targets dim-------',validationTargets.shape);
    #deriving features data, targets data for testing set.    
    testFeatureData,testTargets=deriveData(testData,featureLength);
    print('--------training feature data dim-------',testFeatureData.shape);
    print('--------training targets dim-------',testTargets.shape);   
    #applying the logistic regression
    logisticPredictedWeights=logisticRegression(trainingFeatureData,trainingTargets);
    #computing accuracy using the predicted weights for training,validation and test.
    logisticTrainingAccuracy=computeAccuracy(logisticPredictedWeights,trainingFeatureData,trainingTargets);
    logisticValidationAccuracy=computeAccuracy(logisticPredictedWeights,validationFeatureData,validationTargets);
    logisticTestingAccuracy=computeAccuracy(logisticPredictedWeights,testFeatureData,testTargets)
    
    print('Training Accuracy: ',logisticTrainingAccuracy)
    print('Validation Accuracy: ',logisticValidationAccuracy);
    print('Test Accuracy: ',logisticTestingAccuracy);
    
    
    
    
    
    #neural network
    model=initializeModel(trainingFeatureData);
    history=oprateNeuralNetwork(model,trainingFeatureData,validationFeatureData,trainingTargets,validationTargets); 
    #printKerasGraphs(history);
    metric=evaluateNeuralNetwork(model,testFeatureData,testTargets)
    print('----neural network accuracy-------',metric *100)
    
    
    #compute the centroid list by kmeans
    centroidList=clusterData(trainingFeatureData);   
    varianceVector=generateVarianceVector(trainingFeatureData);
    #getting big sigma matrix
    bigSigma=generatingCoVarianceMatrix(varianceVector,trainingFeatureData.shape[1]);
    print('--------big sigma----------',bigSigma.shape)
    modifyingMergedData(bigSigma,[]);
    #print(bigSigma);
    #computing the inverse of the bigsigma
    bigSigmaInverse=np.linalg.inv(bigSigma);
    #creating design matrix for training
    designMatrixTraining=generateDesignMatrix(bigSigmaInverse,centroidList,trainingFeatureData);
    print('--------training design matrix dim-------',designMatrixTraining.shape);  
    #creating design matrix for validation
    designMatrixValidation=generateDesignMatrix(bigSigmaInverse,centroidList,validationFeatureData);
    print('--------validation design matrix dim-------',designMatrixValidation.shape);
    #creating design matrix for testing
    designMatrixTesting=generateDesignMatrix(bigSigmaInverse,centroidList,testFeatureData);
    print('--------testing design matrix dim-------',designMatrixTesting.shape);
    #applying gradient descent for the linear regression
    predictedWeights=applyGradientDescent(trainingTargets,validationTargets,testTargets,designMatrixTraining,designMatrixValidation,designMatrixTesting);

In [46]:
for i in range(1,5,1):
    projectKnob(3);

--------merge data dim------- (20000, 513)
--------random merge data dim------- (20000, 513)
--------training data dim------- (16000, 513)
--------validation data dim------- (2000, 513)
--------testData data dim------- (2000, 513)
--------training feature data dim------- (16000, 512)
--------training targets dim------- (16000, 1)
--------training feature data dim------- (2000, 512)
--------training targets dim------- (2000, 1)
--------training feature data dim------- (2000, 512)
--------training targets dim------- (2000, 1)
(512, 1)
Training Accuracy:  57.0125
Validation Accuracy:  56.05
Test Accuracy:  53.05
Train on 15300 samples, validate on 2700 samples
Epoch 1/25
15300/15300 [==============================] - 11s 713us/step - loss: 0.6057 - acc: 0.6546 - val_loss: 0.4812 - val_acc: 0.7752
Epoch 2/25
15300/15300 [==============================] - 8s 540us/step - loss: 0.4093 - acc: 0.8190 - val_loss: 0.4016 - val_acc: 0.8215
Epoch 3/25
15300/15300 [==============================] -

TypeError: 'numpy.float64' object is not iterable

In [ ]:
ee=hh[:,1:2]

In [ ]:
ee[0][0][0]

In [ ]:
trainingTargets[2].item()

In [ ]:
import cv2

In [ ]:
cv2.__version__

In [ ]:
merf=np.delete(mergedData,zeroValueColumns,1)

In [ ]:
merf.shape

In [ ]:
math.pow(2,2)

In [ ]:
np.matrix().transpose().shape

In [ ]:
np.zeros(18).shape

In [ ]:
df=

In [ ]:
ss=np.asarray(df)

In [ ]:
xx=ss.transpose();

In [ ]:
xx.shape